In [1]:
from __future__ import print_function
import numpy as np
import awkward
import uproot_methods

import fnal_column_analysis_tools.lookup_tools as lookup_tools
from fnal_column_analysis_tools.analysis_objects import JaggedDecoratedFourVector, JaggedWithLorentz

In [2]:
# import a bunch of correction histograms
# wrapped in numpythonic interface
weights_ext = lookup_tools.extractor()
correction_data = open("newCorrectionFiles.txt").readlines()
weights_ext.add_weight_sets(correction_data)
weights_ext.finalize()
weights_eval = weights_ext.make_evaluator()
#what scale factors / corrections do we have available? (uncomment to dump this)
#dir(weights_eval)

In [3]:
#let's create some fake data
np.random.seed(4)
nrows = 4000000

nwide = 4

counts1 = np.minimum(np.random.exponential(0.5, size=nrows).astype(int), 20)
counts2 = np.minimum(np.random.exponential(0.5, size=nrows).astype(int), 20)
content1_pxy = np.random.normal(loc=20.0,scale=5.0,size=np.sum(counts1)*2).reshape((-1, 2))
content2_pxy = np.random.normal(loc=20.0,scale=5.0,size=np.sum(counts2)*2).reshape((-1, 2))
content1_z = np.random.normal(loc=0, scale=55, size=np.sum(counts1))
content2_z = np.random.normal(loc=0, scale=55, size=np.sum(counts2))
awk_vector1 = JaggedWithLorentz.fromcounts(counts1,uproot_methods.TLorentzVectorArray(content1_pxy[:,0],
                                                                                      content1_pxy[:,1],
                                                                                      content1_z[:],
                                                                                      np.sqrt(content1_pxy[:,0]**2 + 
                                                                                              content1_pxy[:,1]**2 + 
                                                                                              content1_z[:]**2 )))
awk_vector2 = JaggedWithLorentz.fromcounts(counts2,uproot_methods.TLorentzVectorArray(content2_pxy[:,0],
                                                                                      content2_pxy[:,1],
                                                                                      content2_z[:],
                                                                                      np.sqrt(content2_pxy[:,0]**2 + 
                                                                                              content2_pxy[:,1]**2 + 
                                                                                              content2_z[:]**2 )))

In [4]:
#make the basic tables of "muons" and "electrons"
muons = JaggedDecoratedFourVector.fromcounts(counts1,
                                             p4 = awk_vector1.content
                                             )
electrons = JaggedDecoratedFourVector.fromcounts(counts2,
                                                 p4 = awk_vector2.content
                                                 )

muon_eta = muons.p4.eta
muon_pt = muons.p4.pt

electron_eta = electrons.p4.eta
electron_pt = electrons.p4.pt

In [5]:
#using the lorentz vectors
%timeit temp1 = weights_eval["muScaleFactor_MediumId_Iso"](np.abs(muons.p4.eta),muons.p4.pt)
%timeit temp2 = weights_eval["eleScaleFactor_MediumId_MVA_POG"](electrons.p4.eta,electrons.p4.pt)

# pt and eta jagged arrays, no internal eval
%timeit temp1 = weights_eval["muScaleFactor_MediumId_Iso"](np.abs(muon_eta),muon_pt)
%timeit temp2 = weights_eval["eleScaleFactor_MediumId_MVA_POG"](electron_eta,electron_pt)

1 loop, best of 3: 3.01 s per loop
1 loop, best of 3: 3.07 s per loop
10 loops, best of 3: 70.4 ms per loop
10 loops, best of 3: 34.1 ms per loop
